In [27]:
import pandas as pd
import numpy as np

df = pd.read_csv('/Users/Mirzafamily/Desktop/DS311-Group-Project-Documentation/sport/fifa_worldcup.csv')

In [13]:
#NA Report
ls = list(df.columns)
for x in ls:
  if sum(df[x].isna()) > 0:
    print(x)
    print(sum(df[x].isna()))
    print()

home_team_goalkeeper_score
15542

away_team_goalkeeper_score
15826

home_team_mean_defense_score
16134

home_team_mean_offense_score
15411

home_team_mean_midfield_score
15759

away_team_mean_defense_score
16357

away_team_mean_offense_score
15609

away_team_mean_midfield_score
15942



In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23921 entries, 0 to 23920
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   date                           23921 non-null  object 
 1   home_team                      23921 non-null  object 
 2   away_team                      23921 non-null  object 
 3   home_team_continent            23921 non-null  object 
 4   away_team_continent            23921 non-null  object 
 5   home_team_fifa_rank            23921 non-null  int64  
 6   away_team_fifa_rank            23921 non-null  int64  
 7   home_team_total_fifa_points    23921 non-null  int64  
 8   away_team_total_fifa_points    23921 non-null  int64  
 9   home_team_score                23921 non-null  int64  
 10  away_team_score                23921 non-null  int64  
 11  tournament                     23921 non-null  object 
 12  city                           23921 non-null 

In [33]:
df.groupby('neutral_location')['country'].count() #T/F games played in neutral location 

neutral_location
False    17947
True      5974
Name: country, dtype: int64

## Away teams with their corresponding average away defense score

In [82]:
awayd = df.groupby('away_team')['away_team_mean_defense_score'].mean() 
df_away =awayd.to_frame() #Series to Dataframe

away_team_mean_defense_score
away_team                                   
Afghanistan                              NaN
Albania                            72.511905
Algeria                            74.587179
American Samoa                           NaN
Andorra                                  NaN
...                                      ...
Vietnam                                  NaN
Wales                              74.796053
Yemen                                    NaN
Zambia                                   NaN
Zimbabwe                           62.638235

[211 rows x 1 columns]

# Top 10

In [93]:
df_away.sort_values(by = 'away_team_mean_defense_score', ascending = False).head(10)

away_team_mean_defense_score
away_team                                
Brazil                          86.086364
Spain                           85.427193
Italy                           85.124742
Germany                         84.671429
England                         84.640426
France                          84.132258
Argentina                       83.353636
Portugal                        82.896154
Belgium                         81.153409
Netherlands                     80.681250

# Home

In [94]:
homed = df.groupby('home_team')['home_team_mean_defense_score'].mean() 
df_home = homed.to_frame() #Series to Dataframe

In [97]:
df_home.sort_values(by = 'home_team_mean_defense_score',ascending = False).head(10)

home_team_mean_defense_score
home_team                                
Brazil                          85.785185
Spain                           85.585366
Italy                           84.960902
England                         84.733607
Germany                         84.700694
France                          84.047887
Portugal                        82.887705
Argentina                       82.646721
Belgium                         81.304717
Netherlands                     80.776423

# Who's Scored The Most? 


In [106]:
def max_goal(type):
    if type == "sum":
        most_goal = pd.DataFrame(df.groupby(['home_team']).sum())
        most_goal_a = pd.DataFrame(df.groupby(['away_team']).sum())
    if type == "mean":
        most_goal = pd.DataFrame(df.groupby(['home_team']).mean())
        most_goal_a = pd.DataFrame(df.groupby(['away_team']).mean())
    most_goal = pd.DataFrame(most_goal.iloc[0:, [4]]).to_numpy()
    most_goal_a = pd.DataFrame(most_goal_a.iloc[0:, [5]]).to_numpy()
    tot = []
    for idx, i in enumerate(most_goal_a):
        if type == "sum":
            tot.append(int(i+most_goal[idx]))
        if type == "mean":
            tot.append(float((i+most_goal[idx])/2))
    country = df.groupby('home_team')['home_team'].apply(list)
    c = []
    for idx, i in enumerate(country):
        c.append(i[0])
    m_h = max(most_goal)
    m_a = max(most_goal_a)
    m = max(tot)
    j = 0
    j_h = 0
    j_a = 0
    for idx, i in enumerate(c):
        if tot[idx] == m:
            j = idx
        if most_goal[idx] == m_h:
            j_h = idx
        if most_goal_a[idx] == m_a:
            j_a = idx
    if type == "sum":
        print("Team that scored the most goals at Home is:", c[j_h], "with", m_h[0], "goals")
        print("Team that scored the most goals Away is:", c[j_a], "with", m_a[0], "goals")
        print("Team that scored the most goals Overall is:", c[j], "with", m, "goals")
    if type == "mean":
        print("Team that have the highest goals per match at Home is:", c[j_h], "with", m_h[0], "goals")
        print("Team that have the highest goals per match Away is:", c[j_a], "with", m_a[0], "goals")
        print("Team that have the highest goals per match overall is:", c[j], "with", m, "goals")
    return c

def who_scored_the_most_goals():
    c = max_goal("sum")
    print()
    max_goal("mean")
    return c

c = who_scored_the_most_goals()


Team that scored the most goals at Home is: USA with 589 goals
Team that scored the most goals Away is: Brazil with 396 goals
Team that scored the most goals Overall is: Brazil with 954 goals

Team that have the highest goals per match at Home is: New Caledonia with 2.6785714285714284 goals
Team that have the highest goals per match Away is: Vanuatu with 2.314814814814815 goals
Team that have the highest goals per match overall is: New Caledonia with 2.2582046332046333 goals


# Who Has Won The Most? 

In [110]:
def who_win_the_most(c, team, score):
    global data
    win = []
    for i in c:
        win.append(len(df[(df[team] == i) & (df['home_team_result'] == score)]))
    m = max(win)
    j = 0
    for idx, i in enumerate(win):
        if m == i:
            j = idx
    tot_game = len(df[df[team] == c[j]])
    if team == 'home_team':
        print("The team who won the most of match is:", c[j], "with", m, "victory at Home which is", m/tot_game*100, "% victory")
    if team == 'away_team':
        print("The team who won the most of match is:", c[j], "with", m, "victory Away which is", m/tot_game*100, "% victory")
    return win


def best_country(c):
    win_h = who_win_the_most(c, 'home_team', "Win")
    win_a = who_win_the_most(c, 'away_team', "Lose")
    win = []
    for idx, i in enumerate(win_h):
        win.append(i + win_a[idx])
    m = max(win)
    j = 0
    for idx, i in enumerate(win):
        if m == i:
            j = idx
    tot_game = len(df[(df['home_team'] == c[j]) | (df['away_team'] == c[j])])
    print("The team who won the most of match is:", c[j], "with", m, "victory Overall which is", m/tot_game*100, "% victory")

best_country(c)

The team who won the most of match is: USA with 194 victory at Home which is 61.78343949044586 % victory
The team who won the most of match is: Brazil with 120 victory Away which is 60.0 % victory
The team who won the most of match is: Brazil with 301 victory Overall which is 69.5150115473441 % victory


# Average Home/Away Country FIFA Rank

In [112]:
cols = ['home_team_fifa_rank', 'away_team_fifa_rank']
df[cols].mean()

home_team_fifa_rank    77.854688
away_team_fifa_rank    80.797375
dtype: float64